# 04-Exercises

The main task is to try and understand the overall structure of the Projectile Motion code.

This might also involve going back over yesterdays materials too.

Now you are more experienced, reading back over the ''How to Think Like a Computer Scientist' chapters (1-7) would be a good thing to do.

**Exercise**

Change the angle launch angle (from horizontal) of the turtle
- to $30^o$
- to $45^o$
- to $90^o$

Todo : four sprites, standing, jumping, celebration, and death. Set the origin at the axes and make the axes align properly. Bug fixes. Replace the number inputs with an arrow that bounces and then gets longer/shorter for power. Make the sides of the screen actual walls that it bounces off of.

In [67]:
"""A script to simulate the motion of a turtle in free fall."""
import turtle as t
from turtle import TurtleGraphicsError
from math import sin, cos, pi, radians
from PIL import Image
import random

# Constants and initial conditions
dt = 0.1 # time step
g = 9.81  # gravitational acceleration
ax0 = 0.0 # acceleration in x direction
ay0 = -g # acceleration in y direction
x0 = 0.0 # initial x position
y0 = 0.0 # initial y position
scale = 5 # scale for drawing
xaxisRelPos = 5 # x position of the x axis relative to the window (e.g. 3 would be 1/3 of the way up the bottom half of the window)
backgroundImage = "background_1.png" # background image
turtleShape1 = "flying_turtle.png" # turtle shape (in flight)
turtleShape2 = "crashed_turtle.gif" # turtle shape (landed)


def createTarget(ttl) -> tuple:
    """Create a random target and return its position."""
    width = t.window_width()
    height = t.window_height()
    xAxisPosition = -height // xaxisRelPos

    # Randomly determine the target's position
    target_x = random.randint(-width//2 + 50, width//2 - 50)
    target_y = random.randint(xAxisPosition + 20, height//2 - 50)

    # Draw the target
    ttl.penup()
    ttl.color("red")
    ttl.goto(target_x, target_y)
    ttl.dot(20)
    
    return target_x, target_y


def checkHit(x, y, target_x, target_y) -> bool:
    """Check if the turtle hit the target."""
    return abs(x - target_x) < 15 and abs(y - target_y) < 15


def drawAxes(ttl, scale: int) -> None:
    """Draw the x and y axes with ticks."""
    width = t.window_width()
    height = t.window_height()
    xAxisPosition = -height // xaxisRelPos
    
    ttl.speed(0)  # fastest speed
    ttl.hideturtle()
    ttl.penup()

    # Draw the x-axis
    ttl.goto(-width//2, xAxisPosition)
    ttl.pendown()
    ttl.goto(width//2, xAxisPosition)
    ttl.penup()

    # Draw the y-axis
    ttl.goto(0, -height//2)
    ttl.pendown()
    ttl.goto(0, height//2)
    ttl.penup()

    # Drawing ticks on the x-axis
    for x in range(-width//2, width//2 + 1, scale*10):
        ttl.goto(x, xAxisPosition - 5)  # 5 pixels long ticks
        ttl.pendown()
        ttl.goto(x, xAxisPosition + 5)
        ttl.penup()

    # Drawing ticks on the y-axis
    for y in range(-height//2, height//2 + 1, scale*10):
        ttl.goto(-5, y)  # 5 pixels long ticks
        ttl.pendown()
        ttl.goto(5, y)
        ttl.penup()


def notLanded(y: float) -> bool:
    """Check if the turtle has landed."""
    height = t.window_height()
    xAxisPosition = -height // xaxisRelPos / scale
    return y >= xAxisPosition


def envSetup(backgroundImage: str, turtleShape1: str, turtleShape2: str) -> None:
    """Set up the environment for the simulation."""

    # Set up the window
    wn = t.Screen()
    wn.bgpic(backgroundImage)
    wn.setup(width=1.0, height=1.0)
    wn.getcanvas().winfo_toplevel().attributes('-fullscreen', True)
    wn.getcanvas().winfo_toplevel().attributes('-alpha', 1.0)
    
    # Create a turtle object
    ttl = t.Turtle()
    ttl.hideturtle()
    ttl.speed(5)
    ttl.pensize(3)
    ttl.color("White")

    # Register the turtle shapes (as GIFs)
    turtle1_img_original = Image.open(turtleShape1)
    turtle1_img_resized = turtle1_img_original.resize((int(turtle1_img_original.width * 0.5), int(turtle1_img_original.height * 0.5)))
    turtle1_img_resized.save('temp_turtle1.gif', 'GIF')
    wn.register_shape('temp_turtle1.gif')

    turtle2_img_original = Image.open(turtleShape2)
    turtle2_img_resized = turtle2_img_original.resize((int(turtle2_img_original.width * 0.5), int(turtle2_img_original.height * 0.5)))
    turtle2_img_resized.save('temp_turtle2.gif', 'GIF')
    wn.register_shape('temp_turtle2.gif')

    ttl.shape('temp_turtle1.gif') # Set the turtle shape to the flying turtle

    # Return the window and turtle objects
    return wn, ttl


def main() -> None:
    """Run the simulation."""
    global x0, y0, v, angle

    # Set up the environment
    wn, ttl = envSetup(backgroundImage, turtleShape1, turtleShape2)

    score = 0
    highscore = 0

    while True:
        # Reset the turtle's position
        x0, y0 = 0, 0
        ttl.penup()
        ttl.goto(scale * x0, scale * y0)

        # Draw the axes
        drawAxes(ttl, scale)
        
        # Create a target
        target_x, target_y = createTarget(ttl)

        ttl.goto(scale * x0, scale * y0) # Reset the turtle's position
        ttl.showturtle() # Show the turtle

        # Get initial velocity and angle from user
        while True:
            try:
                v = int(t.textinput("Input", "Enter the initial velocity (m/s):"))
                break
            except ValueError:
                t.write("Please enter a valid velocity!", align="center", font=("Arial", 12, "normal"))

        while True:
            try:
                angle = int(t.textinput("Input", "Enter the initial angle (degrees):"))
                break
            except ValueError:
                t.write("Please enter a valid angle!", align="center", font=("Arial", 12, "normal"))

        # Calculate initial velocity components
        vx0 = v * cos(radians(angle))
        vy0 = v * sin(radians(angle))

        # Simulate the motion
        hit = False
        while notLanded(y0):
            x1 = x0 + vx0 * dt
            y1 = y0 + vy0 * dt - 0.5 * g * dt**2
            vx1 = vx0
            vy1 = vy0 - g * dt
        
            ttl.goto(scale * x0, scale * y0)
            ttl.pendown()
            ttl.goto(scale * x1, scale * y1)
            
            if checkHit(scale * x1, scale * y1, target_x, target_y):
                ttl.shape('temp_turtle2.gif')
                score += 1
                hit = True
                break

            x0, y0, vx0, vy0 = x1, y1, vx1, vy1

        if not hit:
            ttl.shape('temp_turtle1.gif')
            if score > highscore:
                highscore = score
            score = 0

        # Display score and highscore
        ttl.penup()
        ttl.goto(-wn.window_width() // 2 + 50, wn.window_height() // 2 - 50)
        ttl.color("white")
        ttl.write(f"Score: {score}  Highscore: {highscore}", align="left", font=("Arial", 12, "normal"))

        # Ask user if they want to play again
        play_again = t.textinput("Play Again?", "Do you want to play again? (yes/no)").lower()
        if play_again != "yes":
            break

        ttl.clear()  # Clear the screen for the next round

    t.bye()

if __name__ == "__main__":
    try:
        main()
    except TurtleGraphicsError as e:
        print(f"There was an issue with the turtle graphics: {str(e)}")
        print("Please check the image paths and formats.")




**Exercise**

alter the loop so that you can record the highest point that the turtle reaches.
*Hint* add a new variable, and update it if the new height is larger than the previous highest.

**Exercise**

If the turtle's initial speed is 30 (m/s), what is the furthest distance it can go?

In [11]:
# hint the code below might be helpful
import math

def degree_to_radian(angle):
    """
    convert degrees to radians
    """
    return angle/360*2*math.pi

angle= degree_to_radian(30)
speed = 30
vx0 = speed*math.cos(angle)
vy0 = speed*math.sin(angle)
print(vx0,vy0)

25.98076211353316 14.999999999999998


**Exercise**

In the notebook I write the condition that the turtle has hit the ground using

```python
while notLanded:
```

what does this do? And why write it like this?

**Exercise**

Customize the animation to your taste.
- try and add a new background.
- try to change the turtle to something of your choice (This might need a little image manipulation).

e.g. try setting it in space and turning gravity upside down

**Exercise**

Add an air resistance term proportional to the velocity to the simulation. 

Change the constant of proportianality and see how it affects your results.

**Exercise**

What would be needed to make this into a game?